In [13]:
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import re, string 
import pandas as pd   
from collections import defaultdict
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from gensim.models import Word2Vec

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [14]:
main_df=pd.read_csv("nlp-getting-started/train.csv")

In [78]:
main_df[main_df['id']==454]

,id,keyword,location,text,target
311,454,armageddon,Wrigley Field,@KatieKatCubs you already know how this shit g...,0


In [18]:
df_text=main_df[['text']]

In [19]:
df_text

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...
...,...
7608,Two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,Police investigating after an e-bike collided ...


In [ ]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
from pycontractions import Contractions
import gensim.downloader as api

#This will give us stop words
#en_default = en_core_web_md
nlp = spacy.load('en_default') 


# Choose model accordingly for contractions function
model = api.load("glove-twitter-25")
# model = api.load("glove-twitter-100")
# model = api.load("word2vec-google-news-300")

cont = Contractions(kv_model=model)
cont.load_models()


In [59]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [91]:

# exclude words from spacy stopwords list
deselect_stop_words = ['no', 'not']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def remove_punctuation_function(text):
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;'+string.punctuation+']')
    text = REPLACE_BY_SPACE_RE.sub('', text)
    return text


def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = list(cont.expand_texts([text], precise=True))[0]
    return text
def remove_user_name(text):
    """Remove the twitter usernames, e.g. @usatoday,@bbcmtd """
    text = re.sub('@[^\s]+', '', text)
    return text

def remove_urls(text):
    """Remove the urls, e.g. http://t.co/lHYXEOHY6C"""
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
    text = re.sub(r'http\S+', '', text)
    return text



def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True,remove_punctuation=True,remove_user=True,remove_url=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if remove_user == True:#remove twitter user names
        text=remove_user_name(text)
    if remove_url == True:#remove the urls from the tweets
        text=remove_urls(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()
    if remove_punctuation == True: #remove punctuation
        text=remove_punctuation_function(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [92]:
sentence="FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/c0p3SEsqWm via @usatoday"
clean_text=text_preprocessing(sentence) 
print(clean_text)

['fedex', 'no', 'longer', 'transport', 'bioterror', 'germ', 'wake', 'anthrax', 'lab', 'mishap']


In [93]:
words_list=[]

In [94]:
for line in df_text.values:
    #print(line[ ])
    words_list.append(text_preprocessing(line[0]))
    #print(clean_text)
    #print('\n')

INFO - 18:46:29: Removed 1 and 0 OOV words from document 1 and 2 (respectively).
INFO - 18:46:29: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:29: built Dictionary(12 unique tokens: ['across', 'an', 'building', 'emergency', 'evacuation']...) from 2 documents (total 24 corpus positions)
INFO - 18:46:29: Removed 1 and 0 OOV words from document 1 and 2 (respectively).
INFO - 18:46:29: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:29: built Dictionary(12 unique tokens: ['across', 'an', 'building', 'emergency', 'evacuation']...) from 2 documents (total 24 corpus positions)
INFO - 18:46:29: Removed 2 and 1 OOV words from document 1 and 2 (respectively).
INFO - 18:46:29: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:29: built Dictionary(3 unique tokens: ['up', 'has', 'what']) from 2 documents (total 4 corpus positions)
INFO - 18:46:29: Removed 2 and 1 OOV words from document 1 and 2 (respectively).
INFO - 18:46:29: adding documen

INFO - 18:46:31: Removed 4 and 3 OOV words from document 1 and 2 (respectively).
INFO - 18:46:31: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:31: built Dictionary(18 unique tokens: ['a', 'annihilated', 'because', 'from', 'great']...) from 2 documents (total 34 corpus positions)
INFO - 18:46:31: Removed 4 and 3 OOV words from document 1 and 2 (respectively).
INFO - 18:46:31: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:31: built Dictionary(17 unique tokens: ['a', 'annihilated', 'because', 'from', 'great']...) from 2 documents (total 34 corpus positions)
INFO - 18:46:32: Removed 8 and 7 OOV words from document 1 and 2 (respectively).
INFO - 18:46:32: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:32: built Dictionary(13 unique tokens: ['after', 'and', 'books', 'current', 'died']...) from 2 documents (total 30 corpus positions)
INFO - 18:46:32: Removed 8 and 7 OOV words from document 1 and 2 (respectively).
INFO - 18:46:32: 

INFO - 18:46:33: Removed 5 and 5 OOV words from document 1 and 2 (respectively).
INFO - 18:46:33: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:33: built Dictionary(15 unique tokens: ['a', 'armageddon', 'bears', 'certainty', 'coat']...) from 2 documents (total 36 corpus positions)
INFO - 18:46:33: Removed 5 and 5 OOV words from document 1 and 2 (respectively).
INFO - 18:46:33: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:33: built Dictionary(15 unique tokens: ['a', 'armageddon', 'bears', 'certainty', 'coat']...) from 2 documents (total 36 corpus positions)
INFO - 18:46:34: Removed 4 and 4 OOV words from document 1 and 2 (respectively).
INFO - 18:46:34: At least one of the documents had no words that were in the vocabulary. Aborting (returning inf).
INFO - 18:46:34: Removed 4 and 4 OOV words from document 1 and 2 (respectively).
INFO - 18:46:34: At least one of the documents had no words that were in the vocabulary. Aborting (returning inf).
IN

INFO - 18:46:37: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:37: built Dictionary(13 unique tokens: ['am', 'and', 'blazing', 'can', 'do']...) from 2 documents (total 24 corpus positions)
INFO - 18:46:37: Removed 3 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:46:37: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:37: built Dictionary(11 unique tokens: ['a', 'and', 'at', 'chicken', 'lip']...) from 2 documents (total 20 corpus positions)
INFO - 18:46:37: Removed 3 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:46:37: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:37: built Dictionary(11 unique tokens: ['a', 'and', 'at', 'chicken', 'lip']...) from 2 documents (total 20 corpus positions)
INFO - 18:46:37: Removed 1 and 1 OOV words from document 1 and 2 (respectively).
INFO - 18:46:37: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:37: built Dictionary(11 unique tokens: ['and'

INFO - 18:46:38: Removed 2 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:46:38: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:38: built Dictionary(18 unique tokens: ['a', 'actually', 'arent', 'blight', 'dwarves']...) from 2 documents (total 41 corpus positions)
INFO - 18:46:38: Removed 2 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:46:38: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:38: built Dictionary(18 unique tokens: ['a', 'actually', 'are', 'blight', 'dwarves']...) from 2 documents (total 43 corpus positions)
INFO - 18:46:38: Removed 2 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:46:38: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:46:38: built Dictionary(18 unique tokens: ['a', 'actually', 'are', 'blight', 'dwarves']...) from 2 documents (total 43 corpus positions)
INFO - 18:46:38: Removed 7 and 6 OOV words from document 1 and 2 (respectively).
INFO - 18:46:38: add

...

INFO - 18:47:56: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:56: built Dictionary(16 unique tokens: ['-->', 'already', 'and', 'biggest', 'for']...) from 2 documents (total 36 corpus positions)
INFO - 18:47:56: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:56: built Dictionary(12 unique tokens: ['a', 'dick', 'hes', 'like', 'never']...) from 2 documents (total 21 corpus positions)
INFO - 18:47:56: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:56: built Dictionary(12 unique tokens: ['a', 'dick', 'hes', 'like', 'never']...) from 2 documents (total 21 corpus positions)
INFO - 18:47:56: Removed 4 and 3 OOV words from document 1 and 2 (respectively).
INFO - 18:47:56: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:56: built Dictionary(13 unique tokens: ['and', 'its', 'lightning', 'missed', 'rly']...) from 2 documents (total 24 corpus positions)
INFO - 18:47:57: Removed 4 and 3 OOV words from document 1 and 2 (

INFO - 18:47:59: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:59: built Dictionary(16 unique tokens: ['and', 'but', 'cat', 'going', 'in']...) from 2 documents (total 32 corpus positions)
INFO - 18:47:59: Removed 5 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:47:59: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:59: built Dictionary(16 unique tokens: ['and', 'but', 'cat', 'going', 'in']...) from 2 documents (total 32 corpus positions)
INFO - 18:47:59: Removed 5 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:47:59: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:59: built Dictionary(15 unique tokens: ['and', 'but', 'cat', 'going', 'in']...) from 2 documents (total 32 corpus positions)
INFO - 18:47:59: Removed 4 and 3 OOV words from document 1 and 2 (respectively).
INFO - 18:47:59: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:47:59: built Dictionary(21 unique tokens: ['a', '

INFO - 18:48:00: Removed 3 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:48:00: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:00: built Dictionary(7 unique tokens: ['know', 'to', 'trouble', 'who', 'you']...) from 2 documents (total 12 corpus positions)
INFO - 18:48:00: Removed 3 and 2 OOV words from document 1 and 2 (respectively).
INFO - 18:48:00: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:00: built Dictionary(7 unique tokens: ['know', 'to', 'trouble', 'who', 'you']...) from 2 documents (total 12 corpus positions)
INFO - 18:48:00: Removed 6 and 5 OOV words from document 1 and 2 (respectively).
INFO - 18:48:00: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:00: built Dictionary(20 unique tokens: ['all', 'are', 'bats', 'defense', 'in']...) from 2 documents (total 40 corpus positions)
INFO - 18:48:00: Removed 6 and 5 OOV words from document 1 and 2 (respectively).
INFO - 18:48:00: adding document #0 to Dic

INFO - 18:48:03: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:03: built Dictionary(7 unique tokens: ['a', 'going', 'like', 'war', 'zone']...) from 2 documents (total 12 corpus positions)
INFO - 18:48:03: Removed 9 and 8 OOV words from document 1 and 2 (respectively).
INFO - 18:48:03: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:03: built Dictionary(19 unique tokens: ['a', 'ahead', 'as', 'best', 'go']...) from 2 documents (total 40 corpus positions)
INFO - 18:48:03: Removed 9 and 8 OOV words from document 1 and 2 (respectively).
INFO - 18:48:03: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:03: built Dictionary(19 unique tokens: ['a', 'ahead', 'as', 'best', 'go']...) from 2 documents (total 40 corpus positions)
INFO - 18:48:03: Removed 9 and 8 OOV words from document 1 and 2 (respectively).
INFO - 18:48:03: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:03: built Dictionary(19 unique tokens: ['a', 'ahea

INFO - 18:48:06: built Dictionary(21 unique tokens: ['a', 'at', 'could', 'go', 'has']...) from 2 documents (total 44 corpus positions)
INFO - 18:48:06: Removed 6 and 5 OOV words from document 1 and 2 (respectively).
INFO - 18:48:06: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:06: built Dictionary(21 unique tokens: ['a', 'at', 'could', 'go', 'has']...) from 2 documents (total 44 corpus positions)
INFO - 18:48:06: Removed 4 and 3 OOV words from document 1 and 2 (respectively).
INFO - 18:48:06: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:06: built Dictionary(12 unique tokens: ['and', 'either', 'helping', 'my', 'surgical']...) from 2 documents (total 22 corpus positions)
INFO - 18:48:06: Removed 4 and 3 OOV words from document 1 and 2 (respectively).
INFO - 18:48:06: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 18:48:06: built Dictionary(12 unique tokens: ['and', 'either', 'helping', 'my', 'surgical']...) from 2 documents (total

In [142]:
words_list

[['deed', 'reason', 'earthquake', 'allah', 'forgive'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['resident',
  'ask',
  'shelter',
  'place',
  'notify',
  'officer',
  'no',
  'evacuation',
  'shelter',
  'place',
  'order',
  'expect'],
 ['people', 'receive', 'wildfire', 'evacuation', 'order', 'california'],
 ['get',
  'send',
  'photo',
  'ruby',
  'alaska',
  'smoke',
  'wildfire',
  'pour',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  'close',
  'direction',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfire'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'cause',
  'flash',
  'flooding',
  'street',
  'manitou',
  'colorado',
  'spring',
  'area'],
 ['hill', 'fire', 'wood'],
 ['emergency', 'evacuation', 'happen', 'building', 'street'],
 ['afraid', 'tornado', 'come', 'area'],
 ['people', 'die', 'heat', 'wave', 'far'],
 ['haha',
  'south',
  'tampa',
  'get',
  'flood',
  'hah',
  'wait',
  'second',
  'live',
  'south',
  'tampa'

In [116]:
#from gensim.models.phrases import Phrases, Phraser
#phrases = Phrases(words_list, min_count=30, progress_per=10000)
model = Word2Vec(words_list, size=100, window=5, min_count=1, workers=4)

INFO - 19:41:48: collecting all words and their counts
INFO - 19:41:48: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:41:48: collected 12881 word types from a corpus of 61632 raw words and 7613 sentences
INFO - 19:41:48: Loading a fresh vocabulary
INFO - 19:41:48: effective_min_count=1 retains 12881 unique words (100% of original 12881, drops 0)
INFO - 19:41:48: effective_min_count=1 leaves 61632 word corpus (100% of original 61632, drops 0)
INFO - 19:41:48: deleting the raw counts dictionary of 12881 items
INFO - 19:41:48: sample=0.001 downsamples 13 most-common words
INFO - 19:41:48: downsampling leaves estimated 60518 word corpus (98.2% of prior 61632)
INFO - 19:41:48: estimated required memory for 12881 words and 100 dimensions: 16745300 bytes
INFO - 19:41:48: resetting layer weights
INFO - 19:41:48: training model with 4 workers on 12881 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO - 19:41:48: worker thread finish

In [112]:
#bigram = Phraser(phrases)
#sentences = bigram[words_list]
model.build_vocab(words_list)

INFO - 19:33:50: collecting all words and their counts
INFO - 19:33:50: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:33:50: collected 12881 word types from a corpus of 61632 raw words and 7613 sentences
INFO - 19:33:50: Loading a fresh vocabulary
INFO - 19:33:50: effective_min_count=1 retains 12881 unique words (100% of original 12881, drops 0)
INFO - 19:33:50: effective_min_count=1 leaves 61632 word corpus (100% of original 61632, drops 0)
INFO - 19:33:50: deleting the raw counts dictionary of 12881 items
INFO - 19:33:50: sample=0.001 downsamples 13 most-common words
INFO - 19:33:50: downsampling leaves estimated 60518 word corpus (98.2% of prior 61632)
INFO - 19:33:50: estimated required memory for 12881 words and 100 dimensions: 16745300 bytes
INFO - 19:33:50: resetting layer weights


In [118]:
#
#model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
word_vectors = model.wv

In [120]:
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

fname = "vectors.kv"
word_vectors.save(fname)
word_vectors = KeyedVectors.load(fname, mmap='r')

INFO - 19:45:54: saving Word2VecKeyedVectors object under vectors.kv, separately None
INFO - 19:45:54: not storing attribute vectors_norm
INFO - 19:45:54: saved vectors.kv
INFO - 19:45:54: loading Word2VecKeyedVectors object from vectors.kv
INFO - 19:45:54: setting ignored attribute vectors_norm to None
INFO - 19:45:54: loaded vectors.kv


In [121]:
from gensim.test.utils import datapath
wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False) 

INFO - 19:47:15: loading projection weights from C:\Users\akash\Anaconda3\lib\site-packages\gensim\test\test_data\word2vec_pre_kv_c
INFO - 19:47:15: loaded (1750, 10) matrix from C:\Users\akash\Anaconda3\lib\site-packages\gensim\test\test_data\word2vec_pre_kv_c


In [138]:
sim = word_vectors.n_similarity(['ablaze'], ['sky'])
print("{:.4f}".format(sim))

0.9288


In [98]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

12891

In [89]:
word_freq

defaultdict(int,
            {'deed': 3,
             'reason': 32,
             'earthquake': 54,
             'allah': 10,
             'forgive': 6,
             'forest': 67,
             'fire': 360,
             'near': 57,
             'la': 26,
             'ronge': 2,
             'sask': 2,
             'canada': 12,
             'resident': 9,
             'ask': 30,
             'shelter': 8,
             'place': 37,
             'notify': 2,
             'officer': 38,
             'no': 255,
             'evacuation': 55,
             'order': 40,
             'expect': 33,
             'people': 200,
             'receive': 5,
             'wildfire': 80,
             'california': 73,
             'get': 213,
             'send': 53,
             'photo': 63,
             'ruby': 2,
             'alaska': 6,
             'smoke': 53,
             'pour': 5,
             'school': 72,
             'rockyfire': 5,
             'update': 54,
             ' ': 1553,
      

In [99]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['not', 'fire', 'like', 'no', 'new', 'go', 'get', 'people', 'news', 'video']

In [100]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

from gensim.models import Word2Vec
w2v_model = Word2Vec(window=2,
                     alpha=0.01,
                     workers=cores-1,
                     iter=100)

In [101]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 18:49:01: collecting all words and their counts
INFO - 18:49:01: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 18:49:01: collected 12891 word types from a corpus of 61218 raw words and 7613 sentences
INFO - 18:49:01: Loading a fresh vocabulary
INFO - 18:49:01: effective_min_count=5 retains 2251 unique words (17% of original 12891, drops 10640)
INFO - 18:49:01: effective_min_count=5 leaves 45879 word corpus (74% of original 61218, drops 15339)
INFO - 18:49:01: deleting the raw counts dictionary of 12891 items
INFO - 18:49:01: sample=0.001 downsamples 31 most-common words
INFO - 18:49:01: downsampling leaves estimated 44111 word corpus (96.1% of prior 45879)
INFO - 18:49:01: estimated required memory for 2251 words and 100 dimensions: 2926300 bytes
INFO - 18:49:01: resetting layer weights


Time to build vocab: 0.01 mins


In [102]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=50, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 18:49:15: training model with 7 workers on 2251 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
INFO - 18:49:15: worker thread finished; awaiting finish of 6 more threads
INFO - 18:49:15: worker thread finished; awaiting finish of 5 more threads
INFO - 18:49:15: worker thread finished; awaiting finish of 4 more threads
INFO - 18:49:15: worker thread finished; awaiting finish of 3 more threads
INFO - 18:49:15: worker thread finished; awaiting finish of 2 more threads
INFO - 18:49:15: worker thread finished; awaiting finish of 1 more threads
INFO - 18:49:15: worker thread finished; awaiting finish of 0 more threads
INFO - 18:49:15: EPOCH - 1 : training on 61218 raw words (44132 effective words) took 0.5s, 92940 effective words/s
INFO - 18:49:16: worker thread finished; awaiting finish of 6 more threads
INFO - 18:49:16: worker thread finished; awaiting finish of 5 more threads
INFO - 18:49:16: worker thread finished; awaiting finish of 4 more threads
I

INFO - 18:49:18: worker thread finished; awaiting finish of 1 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 0 more threads
INFO - 18:49:18: EPOCH - 13 : training on 61218 raw words (44183 effective words) took 0.2s, 206382 effective words/s
INFO - 18:49:18: worker thread finished; awaiting finish of 6 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 5 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 4 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 3 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 2 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 1 more threads
INFO - 18:49:18: worker thread finished; awaiting finish of 0 more threads
INFO - 18:49:18: EPOCH - 14 : training on 61218 raw words (44069 effective words) took 0.2s, 201408 effective words/s
INFO - 18:49:19: worker thread finished; awaiting finish of 6 more threads
INFO - 18:49:1

INFO - 18:49:21: worker thread finished; awaiting finish of 3 more threads
INFO - 18:49:21: worker thread finished; awaiting finish of 2 more threads
INFO - 18:49:21: worker thread finished; awaiting finish of 1 more threads
INFO - 18:49:21: worker thread finished; awaiting finish of 0 more threads
INFO - 18:49:21: EPOCH - 26 : training on 61218 raw words (44156 effective words) took 0.2s, 188577 effective words/s
INFO - 18:49:22: worker thread finished; awaiting finish of 6 more threads
INFO - 18:49:22: worker thread finished; awaiting finish of 5 more threads
INFO - 18:49:22: worker thread finished; awaiting finish of 4 more threads
INFO - 18:49:22: worker thread finished; awaiting finish of 3 more threads
INFO - 18:49:22: worker thread finished; awaiting finish of 2 more threads
INFO - 18:49:22: worker thread finished; awaiting finish of 1 more threads
INFO - 18:49:22: worker thread finished; awaiting finish of 0 more threads
INFO - 18:49:22: EPOCH - 27 : training on 61218 raw words

INFO - 18:49:25: worker thread finished; awaiting finish of 5 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 4 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 3 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 2 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 1 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 0 more threads
INFO - 18:49:25: EPOCH - 39 : training on 61218 raw words (44081 effective words) took 0.2s, 178670 effective words/s
INFO - 18:49:25: worker thread finished; awaiting finish of 6 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 5 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 4 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 3 more threads
INFO - 18:49:25: worker thread finished; awaiting finish of 2 more threads
INFO - 18:49:25: worker thread finished; awaiting finish 

Time to train the model: 0.21 mins


In [140]:
word_vectors.init_sims(replace=True)


INFO - 20:21:56: precomputing L2-norms of word weight vectors


In [104]:
model.save("word2vec.model")

INFO - 18:51:19: saving Word2VecKeyedVectors object under word2vec.model, separately None
INFO - 18:51:19: storing np array 'vectors' to word2vec.model.vectors.npy
INFO - 18:51:20: not storing attribute vectors_norm
INFO - 18:51:25: saved word2vec.model


In [141]:
word_vectors.wv.most_similar(positive=["fire"])

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('not', 0.9988853931427002),
 ('people', 0.998866081237793),
 ('news', 0.9987478256225586),
 ('death', 0.9985353350639343),
 ('come', 0.9985029697418213),
 ('flood', 0.9984305500984192),
 ('new', 0.9984046816825867),
 ('like', 0.9983848333358765),
 ('man', 0.9983129501342773),
 ('emergency', 0.9983068704605103)]